In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import data
obs_A = pd.read_parquet('../../preprocessing/data/obs_A.parquet').drop(columns='date_forecast')
obs_B = pd.read_parquet('../../preprocessing/data/obs_B.parquet').drop(columns='date_forecast')
obs_C = pd.read_parquet('../../preprocessing/data/obs_C.parquet').drop(columns='date_forecast')
est_A = pd.read_parquet('../../preprocessing/data/est_A.parquet').drop(columns='date_forecast')
est_B = pd.read_parquet('../../preprocessing/data/est_B.parquet').drop(columns='date_forecast')
est_C = pd.read_parquet('../../preprocessing/data/est_C.parquet').drop(columns='date_forecast')
test_A = pd.read_parquet('../../preprocessing/data/test_A.parquet').drop(columns='date_forecast')
test_B = pd.read_parquet('../../preprocessing/data/test_B.parquet').drop(columns='date_forecast')
test_C = pd.read_parquet('../../preprocessing/data/test_C.parquet').drop(columns='date_forecast')

# Columns to drop
# columns = [
#     'date_forecast',
#     'super_cooled_liquid_water:kgm2',
#     'air_density_2m:kgm3',
#     'snow_water:kgm2',
#     'precip_5min:mm',
#     'precip_type_5min:idx',
#     'rain_water:kgm2',
#     'snow_melt_10min:mm',
#     'dew_or_rime:idx',
#     'snow_depth:cm',
#     'prob_rime:p',
#     'is_day:idx',
#     'is_in_shadow:idx',
#     'visibility:m',
#     'relative_humidity_1000hPa:p',
#     'temp_dewpoint_diff'
# ]

# Drop columns
# obs_A = obs_A.drop(columns=columns)
# obs_B = obs_B.drop(columns=columns)
# obs_C = obs_C.drop(columns=columns)
# est_A = est_A.drop(columns=columns)
# est_B = est_B.drop(columns=columns)
# est_C = est_C.drop(columns=columns)
# test_A = test_A.drop(columns=columns)
# test_B = test_B.drop(columns=columns)
# test_C = test_C.drop(columns=columns)

In [3]:
# Concatinate
A = pd.concat([obs_A, est_A])
B = pd.concat([obs_B, est_B])
C = pd.concat([obs_C, est_C])

# Data splits for submissions
# X_A = A.drop(columns='pv_measurement')
# y_A = A['pv_measurement']
# X_B = B.drop(columns='pv_measurement')
# y_B = B['pv_measurement']
# X_C = C.drop(columns='pv_measurement')
# y_C = C['pv_measurement']

# Data splits for testing
train_A, test_A = train_test_split(A, test_size=0.2, shuffle=True, random_state=42)
X_train_A = train_A.drop(columns='pv_measurement')
y_train_A = train_A['pv_measurement']
X_test_A = test_A.drop(columns='pv_measurement')
y_test_A = test_A['pv_measurement']

train_B, test_B = train_test_split(B, test_size=0.2, shuffle=True, random_state=42)
X_train_B = train_B.drop(columns='pv_measurement')
y_train_B = train_B['pv_measurement']
X_test_B = test_B.drop(columns='pv_measurement')
y_test_B = test_B['pv_measurement']

train_C, test_C = train_test_split(C, test_size=0.2, shuffle=True, random_state=42)
X_train_C = train_C.drop(columns='pv_measurement')
y_train_C = train_C['pv_measurement']
X_test_C = test_C.drop(columns='pv_measurement')
y_test_C = test_C['pv_measurement']



In [4]:
# Inspect data
X_train_A.describe()

,snow_accumulation,total_radiation,sfc_pressure:hPa,month,year,date_forecast_fft_amplitude,date_forecast_fft_phase,sun_elevation:d_fft_amplitude,sun_elevation:d_fft_phase,clear_sky_rad:W_rate_of_change,...,pressure_gradient,temp_dewpoint_diff,sun_elevation:d_rolling_avg_6,super_cooled_liquid_water:kgm2,relative_humidity_1000hPa:p,is_day:idx,is_in_shadow:idx,pressure_50m:hPa,sun_azimuth:d,absolute_humidity_2m:gm3
count,27233.000000,27233.000000,27233.000000,27233.000000,27233.000000,2.723300e+04,27233.000000,27233.000000,27233.000000,27233.000000,...,27233.000000,27233.000000,27233.000000,27233.000000,27233.000000,27233.000000,27233.000000,27233.000000,27233.000000,27233.000000
mean,0.333873,95.155769,1008.354736,6.541475,2020.866889,5.915843e+17,-0.015143,171.071385,-0.000939,-0.003435,...,-6.117104,3.964125,0.074375,0.055706,73.935188,0.500110,0.533691,1002.210571,179.460312,6.324912
std,1.911448,159.782471,28.889471,3.474102,1.195745,4.515002e+19,2.398508,3219.148150,1.169490,57.838531,...,0.230385,2.659178,22.809958,0.104148,14.027186,0.486049,0.487005,28.721622,97.663559,2.796002
min,0.000000,0.000000,0.000000,1.000000,2019.000000,7.952400e+15,-3.141593,0.000000,-3.141593,-131.825012,...,-6.949951,-1.325012,-46.527250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1001.099976,3.000000,2020.000000,5.498362e+16,-2.360880,16.611680,-0.688569,-10.849998,...,-6.224976,1.875000,-16.124542,0.000000,64.675003,0.000000,0.000000,995.000000,94.081749,4.200000
50%,0.000000,2.200000,1009.900024,7.000000,2021.000000,6.897861e+16,-1.580040,21.725493,0.025956,0.000000,...,-6.100037,3.700012,0.063042,0.000000,76.349998,0.500000,1.000000,1003.799988,180.042252,5.850000
75%,0.000000,128.074997,1017.799988,10.000000,2022.000000,1.255989e+17,2.347538,45.024365,0.688793,9.825012,...,-6.000000,5.600006,16.429041,0.100000,85.174995,1.000000,1.000000,1011.650024,264.975494,8.250000
max,52.825005,773.275024,1043.724976,12.000000,2023.000000,7.399907e+21,3.141540,378968.726443,3.141593,132.575012,...,0.000000,19.025024,46.369333,1.300000,100.000000,1.000000,1.000000,1037.250000,348.486511,17.350000


In [5]:
# Initalize the models
parameters              = {
                            'colsample_bytree': 0.8, 
                            'gamma': 0.8, 
                            'learning_rate': 0.008, 
                            'max_depth': 10, 
                            'min_child_weight': 10, 
                            'n_estimators': 550, 
                            'reg_alpha': 1, 
                            'reg_lambda': 3, 
                            'subsample': 0.912,
                            'random_state': 0, 
                            'booster': 'gbtree'
                        }

tscv = TimeSeriesSplit(n_splits=10)

model_A = xgb.XGBRegressor(**parameters, n_jobs=-1)
model_B = xgb.XGBRegressor(**parameters, n_jobs=-1)
model_C = xgb.XGBRegressor(**parameters, n_jobs=-1)

In [6]:
# Models for submission
# model_A.fit(X_A, y_A)
# model_B.fit(X_B, y_B)
# model_C.fit(X_C, y_C)

# Models for testing
model_A.fit(X_train_A, y_train_A)
model_B.fit(X_train_B, y_train_B)
model_C.fit(X_train_C, y_train_C)

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.8, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.008, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=10, missing=nan, monotone_constraints=None,
             n_estimators=550, n_jobs=-1, num_parallel_tree=None,
             predictor=None, random_state=0, ...)

In [7]:
# Evaluate
print('MAE A:', mean_absolute_error(y_test_A, model_A.predict(X_test_A)))
print('MAE B:', mean_absolute_error(y_test_B, model_B.predict(X_test_B)))
print('MAE C:', mean_absolute_error(y_test_C, model_C.predict(X_test_C)))

# Total MAE for all three locations
print('Total MAE:', (mean_absolute_error(y_test_A, model_A.predict(X_test_A)) + mean_absolute_error(y_test_B, model_B.predict(X_test_B)) + mean_absolute_error(y_test_C, model_C.predict(X_test_C)))/3)

# MAE with angled features
# MAE A: 167.63315287963007
# MAE B: 24.506774509584318
# MAE C: 22.521968709996468
# Total MAE: 71.55396536640362

# MAE without angled features
# MAE A: 166.3593882945251
# MAE B: 24.566444487584228
# MAE C: 22.3112321535254
# Total MAE: 71.07902164521157

MAE A: 167.4029278327981
MAE B: 24.62271999123831
MAE C: 22.37050570134159
Total MAE: 71.46538450845934


In [12]:
# Feature importance
feature_importances = model_C.feature_importances_
feature_importances = pd.DataFrame({'feature': list(X_train_A.columns), 'importance': feature_importances}).sort_values('importance', ascending = False)

# Print feature importance
for i in range(feature_importances.shape[0]):
    print(f"{i} {feature_importances.iloc[i, 0]}: {feature_importances.iloc[i, 1]}")

0 temp_rad_interaction: 0.546764612197876
1 total_radiation: 0.20254617929458618
2 sun_elevation_direct_rad_interaction: 0.06622329354286194
3 clear_sky_rad:W: 0.032762136310338974
4 sun_elevation:d: 0.03222992271184921
5 clear_sky_energy_1h:J: 0.011400318704545498
6 t_1000hPa:K_rolling_avg_24: 0.007054681424051523
7 diffuse_rad:W: 0.00560364406555891
8 t_1000hPa:K: 0.004713616333901882
9 sfc_pressure:hPa_lag_8: 0.004022541455924511
10 msl_pressure:hPa_rolling_avg_24: 0.003530699061229825
11 relative_humidity_1000hPa:p_lag_-3: 0.0033215219154953957
12 pressure_50m:hPa: 0.0031789997592568398
13 snow_accumulation: 0.0030806150753051043
14 temp_dewpoint_diff_lag_-4: 0.0030450276099145412
15 direct_rad_1h:J: 0.002858550986275077
16 sfc_pressure:hPa: 0.002787392819300294
17 dew_point_2m:K_lag_19: 0.0026964587159454823
18 msl_pressure:hPa_lag_3: 0.0025247943121939898
19 total_cloud_cover:p: 0.002392138820141554
20 dew_or_rime:idx_lag_11: 0.0023350731935352087
21 t_1000hPa:K_lag_4: 0.00220583

In [9]:
# Create submission

output_file = 'submission.csv'

pred_A = model_A.predict(test_A)
pred_B = model_B.predict(test_B)
pred_C = model_C.predict(test_C)

pred_A = np.clip(pred_A, 0, None)
pred_B = np.clip(pred_B, 0, None)
pred_C = np.clip(pred_C, 0, None)

# Concatenate predictions
predictions = np.concatenate([pred_A, pred_B, pred_C])

# Create an id array
ids = np.arange(0, len(predictions))

# Create a DataFrame
df = pd.DataFrame({
    'id': ids,
    'prediction': predictions
})

# Save to CSV
df.to_csv(output_file, index=False)
print(f"Submission saved to {output_file}")

ValueError: Feature shape mismatch, expected: 59, got 60

# Features in the best prediction 154 MAE

0 total_radiation: 0.4234558045864105 \
1 direct_rad:W: 0.2128763496875763 \
2 direct_rad_1h:J: 0.0615650974214077 \
3 total_radiation_rolling_avg_3: 0.03578149154782295 \
4 diffuse_rad:W: 0.02013232931494713 \
5 clear_sky_rad:W: 0.013245003297924995 \
6 snow_accumulation: 0.012912849895656109 \
7 sun_azimuth:d_lag_7: 0.010069170035421848 \
8 sun_elevation:d: 0.009076440706849098 \
9 relative_humidity_1000hPa:p_lag_-3: 0.006535227410495281 \
10 sun_azimuth:d: 0.005778656806796789 \
11 diffuse_rad_1h:J: 0.00557068781927228 \
12 temp_dewpoint_diff_lag_-4: 0.0054164547473192215 \
13 clear_sky_rad:W_rate_of_change: 0.005305740050971508 \
14 effective_cloud_cover:p: 0.005097666289657354 \
15 hour: 0.005087343044579029 \
16 date_forecast_fft_amplitude: 0.004949505440890789 \
17 clear_sky_energy_1h:J: 0.004622776992619038 \
18 msl_pressure:hPa_lag_3: 0.004557096865028143 \
19 t_1000hPa:K_rolling_avg_24: 0.004549496807157993 \
20 total_cloud_cover:p: 0.004411020781844854 \
21 visibility:m_lag_-2: 0.00400067213922739 \
22 total_radiation_rate_of_change: 0.003874964313581586 \
23 month: 0.0038293872494250536 \
24 sfc_pressure:hPa: 0.0038288664072752 \
25 average_wind_speed: 0.0038149317260831594 \
26 t_1000hPa:K_lag_4: 0.003787972265854478 \
27 sun_elevation:d_fft_phase: 0.0037678431253880262 \
28 pressure_50m:hPa: 0.0037581848446279764 \
29 msl_pressure:hPa_rolling_avg_24: 0.00369848869740963 \
30 dew_point_2m:K_lag_19: 0.0035477483179420233 \
31 t_1000hPa:K: 0.0035387056414037943 \
32 pressure_100m:hPa: 0.003512951312586665 \
33 total_cloud_cover:p_rolling_avg_6: 0.003464051755145192 \
34 direct_rad:W_rate_of_change: 0.003448427189141512 \
35 dew_point_2m:K: 0.003425233531743288 \
36 absolute_humidity_2m:gm3_rolling_avg_24: 0.003379648318514228 \
37 pressure_gradient: 0.0033784068655222654 \
38 sfc_pressure:hPa_lag_8: 0.003355256747454405 \
39 sun_elevation:d_fft_amplitude: 0.0033312165178358555 \
40 year: 0.003244817489758134 \
41 wind_vector_magnitude: 0.003148019313812256 \
42 date_forecast_fft_phase: 0.003074726788327098 \
43 absolute_humidity_2m:gm3: 0.003025446319952607 \
44 msl_pressure:hPa: 0.003022641409188509 \
45 t_1000hPa:K_rate_of_change: 0.003010170767083764 \
46 effective_cloud_cover:p_rate_of_change: 0.0029358689207583666 \
47 clear_sky_rad:W_rate_of_change_of_change: 0.002842330140992999 \
48 effective_cloud_cover:p_rolling_avg_6: 0.002791582839563489 \
49 clear_sky_rad:W_rolling_avg_6: 0.002718282165005803 \
50 total_cloud_cover:p_rate_of_change: 0.0026860570069402456 \
51 observed: 0.0026119498070329428 \
52 wind_vector_magnitude_lag_8: 0.002609953982755542 \
53 temp_dewpoint_diff: 0.002551795681938529 \
54 total_radiation_rate_of_change_of_change: 0.0025030174292623997 \
55 direct_rad:W_rate_of_change_of_change: 0.00249316799454391 \
56 relative_humidity_1000hPa:p: 0.00249219941906631 \
57 total_cloud_cover:p_rate_of_change_of_change: 0.002429977525025606 \
58 diffuse_rad:W_rate_of_change: 0.0024279451463371515 \
59 sun_elevation:d_rolling_avg_6: 0.0023145817685872316 \
60 visibility:m: 0.002289965283125639 \
61 dew_point_2m:K_rate_of_change: 0.0022229659371078014 \
62 dew_point_2m:K_rate_of_change_of_change: 0.002219223417341709 \
63 effective_cloud_cover:p_rate_of_change_of_change: 0.002212965628132224 \
64 dew_or_rime:idx_lag_11: 0.0021920606959611177 \
65 t_1000hPa:K_rate_of_change_of_change: 0.0020750060211867094 \
66 diffuse_rad:W_rate_of_change_of_change: 0.0020208051428198814 \
67 is_in_shadow:idx: 7.590975292259827e-05 \
68 is_day:idx: 1.739487561280839e-05 \
69 prob_rime:p: 0.0 \